# Complicance Checks
This notebook ensures that the `UncertaintyForest` estimator is `sklearn`-compliant. These checks will be incorporated into tests later.

In [ ]:
import numpy as np
from uncertainty_forest.uncertainty_forest import UncertaintyForest
from sklearn.utils.estimator_checks import check_estimator

In [ ]:
# Check for one class data.
uf = UncertaintyForest(parallel = False)

n = 30
d = 10

X_train = np.random.multivariate_normal(np.ones(d), np.eye(d), n)
y_train = np.repeat("one", n)
print(X_train.shape)
print(y_train.shape)

X_eval = np.array([c*np.ones(d) for c in range(-2, 3)])
print(X_eval)

In [ ]:
uf.fit(X_train, y_train)
cond_probability = uf.predict_proba(X_eval)
y_predicted = uf.predict(X_eval)
print("P(Y|X = x) for x in X_eval:")
print(cond_probability)
print("y_hat for x in X_eval:")
print(y_predicted)

In [ ]:
check_estimator(UncertaintyForest)